In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import time

In [4]:
browser = webdriver.Chrome()
browser.get("https://www.amazon.in/gp/bestsellers/?ref_=nav_em_cs_bestsellers_0_1_1_2")

In [5]:
html_content = browser.page_source
doc = BeautifulSoup(html_content, "html.parser")

In [29]:
browser = webdriver.Edge()
browser.maximize_window()
browser.get("https://www.amazon.in/gp/bestsellers/?ref_=nav_em_cs_bestsellers_0_1_1_2")

html_content = browser.page_source
doc = BeautifulSoup(html_content, "html.parser")
data = []

# Getting all Departments
department_names = []
department_urls = []
base = "https://www.amazon.in"
department_tags = doc.find_all("div", {"role": "group"})[0].find_all("a")
for i in range(0, len(department_tags)):
    department_names.append(department_tags[i].text.strip())
    department_urls.append(base + department_tags[i]["href"])

# Getting Categories
for i in range(0, 1):
    print(f"Scraping Department {i + 1} {department_names[i]}")
    browser.get(department_urls[i])
    html_content = browser.page_source
    doc = BeautifulSoup(html_content, "html.parser")

    category_tags = doc.find_all("div", {"role": "group"})[0].find_all("a")
    special = base + category_tags[0]["href"]
    for j in range(0, 1):
        print(f"Scraping Category {j + 1} {category_tags[j].text.strip()}")
        category_name = category_tags[j].text.strip()
        category_url = base + category_tags[j]["href"]

        # Getting Sub Categories
        browser.get(category_url)
        html_content = browser.page_source
        doc = BeautifulSoup(html_content, "html.parser")

        sub_category_tags = doc.find_all("div", {"role": "group"})[0].find_all("a")
        for k in range(0, 1):
            if special in base + sub_category_tags[0]["href"]:
                break
            else:
                print(f"Scraping Sub Category {k + 1} {sub_category_tags[k].text.strip()}")
                sub_category_name = sub_category_tags[k].text.strip()
                sub_category_url = base + sub_category_tags[k]["href"]

                # Getting Product Links
                browser.get(sub_category_url)
                product_links = []
                
                for l in range(0, 2):
                    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(2)
                    html_content = browser.page_source
                    doc = BeautifulSoup(html_content, "html.parser")
    
                    product_tags = doc.find_all("div", {"id": "gridItemRoot"})
                    # print(len(product_tags))
                    for m in range(0, len(product_tags)):
                        product_links.append(base + product_tags[m].find("a")["href"])
    
                    # Going to next page
                    if l == 1:
                        break
                    try:
                        next_page_tag = doc.find_all("div", {"class": "a-text-center"})[-1].find("li", {"class": "a-last"}).find("a")
                        browser.get(base + next_page_tag["href"])
                    except:
                        pass

                # Getting Info of each product
                for l in range(0, 30):
                    print(f"Scraping Product {l + 1}")
                    browser.get(product_links[l])
                    html_content = browser.page_source
                    doc = BeautifulSoup(html_content, "html.parser")
                    product_data = []

                    product_name = doc.find("span", {"id": "productTitle"}).text.strip()
                    try:
                        price_div = doc.find("div", {"class": "a-section a-spacing-none aok-align-center aok-relative"})
                        product_price = price_div.find("span", {"class": "a-price-whole"}).text.strip()
                        sale_discount = price_div.find("span", {"class": "a-size-large a-color-price savingPriceOverride aok-align-center reinventPriceSavingsPercentageMargin savingsPercentage"}).text.strip().split("-")[1]
                    except:
                        product_price = None
                        sale_discount = None
                        
                    best_seller_rating = doc.find("div", {"class": "a-section a-spacing-base brand-snapshot-flex-row"}).text.strip().split("%")[0] + "%"
                    try:
                        ship_from = doc.find("div", {"id": "tabular-buybox"}).find_all("div", {"tabular-attribute-name": "Ships from"})[1].text.strip()
                    except:
                        ship_from = None
                    
                    sold_by = doc.find("div", {"class": "a-section a-spacing-medium brand-snapshot-flex-row"}).find("span").text.strip()
                    rating = doc.find("span", {"id": "acrPopover"}).find("a").find("span").text.strip()
                    product_description = doc.find_all("div", {"id": "aplus"})[-1].text.strip().replace("Product Description", "").replace("\n", "").replace("  ", " ")
                    try:
                        number_of_bought = doc.find("span", {"id": "social-proofing-faceout-title-tk_bought"}).find("span").text.strip().split("+")[0].replace("K", "000")
                    except:
                        number_of_bought = None

                    data.append([product_name, product_price, sale_discount, best_seller_rating, ship_from, sold_by, rating, product_description, number_of_bought, department_names[i]])

# print(data)
df = pd.DataFrame(data, columns=["Product Name", "Product Price", "Sale Discount", "Best Seller Rating", "Ship From", "Sold by", "Rating", "Product Description", "Number of Bought", "Category"])
df = df.sort_values(by="Sale Discount", ascending=False)
df.to_csv("data.csv", index=None)

browser.close()

Scraping Department 1 Amazon Launchpad
Scraping Category 1 Body
Scraping Sub Category 1 Beauty
Scraping Product 1
Scraping Product 2
Scraping Product 3
Scraping Product 4
Scraping Product 5
Scraping Product 6
Scraping Product 7
Scraping Product 8
Scraping Product 9
Scraping Product 10
Scraping Product 11
Scraping Product 12
Scraping Product 13
Scraping Product 14
Scraping Product 15
Scraping Product 16
Scraping Product 17
Scraping Product 18
Scraping Product 19
Scraping Product 20
Scraping Product 21
Scraping Product 22
Scraping Product 23
Scraping Product 24
Scraping Product 25
Scraping Product 26
Scraping Product 27
Scraping Product 28
Scraping Product 29
Scraping Product 30


<h2>Info of each product</h2>

In [17]:
browser = webdriver.Edge()
browser.get("https://www.amazon.in/Niacinamide-Multi-Peptide-Hyaluronic-Correction-Hyperpigmentation/dp/B097KSJ6LR/ref=zg_bs_g_10894225031_d_sccl_2/262-3973435-7950060?th=1")
html_content = browser.page_source
doc = BeautifulSoup(html_content, "html.parser")

best_seller_rating = doc.find("div", {"class": "a-section a-spacing-base brand-snapshot-flex-row"}).text.strip().split("%")[0] + "%"

try:
    ships_tag = doc.find("div", {"id": "tabular-buybox"}).find_all("div", {"tabular-attribute-name": "Ships from"})[1].text.strip()
except:
    ships_tag = None
print(best_seller_rating, ships_tag)

sold_by = doc.find("div", {"class": "a-section a-spacing-medium brand-snapshot-flex-row"}).find("span").text.strip()
print(sold_by)

rating = doc.find("span", {"id": "acrPopover"}).find("a").find("span").text.strip()
print(rating)

product_description = doc.find_all("div", {"id": "aplus"})[1].text.strip().replace("Product Description", "")
print(product_description)

number_of_bought = doc.find("span", {"id": "social-proofing-faceout-title-tk_bought"}).find("span").text.strip().split("+")[0].replace("K", "000")
print(number_of_bought)

87% Amazon
COS-IQ
4.0

2000
